In [1]:
import pandas as pd
from sqlalchemy import create_engine

import psycopg2
import matplotlib.pyplot as plt
from configparser import ConfigParser


Extract CSV files into DataFrames

In [2]:
netflix_file = "netflix_data_clean.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
1,20,80060297,Movie,Manhattan Romance,Tom O'Brien,"Tom O'Brien, Katherine Waterston, Caitlin Fitz...",United States,"September 8, 2017",2014,TV-14,98 min,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...
2,23,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,"September 8, 2017",2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f..."
3,24,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,"September 8, 2017",2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...
4,25,80045922,Movie,6 Years,Hannah Fidell,"Taissa Farmiga, Ben Rosenfield, Lindsay Burdge...",United States,"September 8, 2015",2015,NR,80 min,"Dramas, Independent Movies, Romantic Movies",As a volatile young couple who have been toget...


In [3]:
omdb_file = "omdb_data_clean.csv"
omdb_df = pd.read_csv(omdb_file)
omdb_df.head()

,Unnamed: 0,title,genre,language,awards,metascore,boxoffice,production,imdbrating,imdbvotes,imdbid
0,1,Manhattan Romance,"Comedy, Drama, Romance",English,3 wins.,NaN,NaN,Beacon Films Inc.,5.5,"1,122",tt2608324
1,2,Stonehearst Asylum,"Drama, Horror, Thriller",English,1 win & 1 nomination.,52.0,NaN,Millenium Entertainment,6.8,"48,493",tt1772264
2,3,The Runner,Drama,English,NaN,39.0,NaN,Alchemy,4.7,"4,768",tt3687398
3,4,6 Years,"Drama, Romance",English,4 nominations.,53.0,NaN,NaN,5.6,"10,784",tt3799372
4,5,Sierra Burgess Is a Loser,"Comedy, Drama, Romance",English,2 nominations.,60.0,NaN,Netflix,5.9,"31,443",tt3120280


In [4]:
tmdb_file = "tmdb_data_clean.csv"
tmdb_df = pd.read_csv(tmdb_file)
tmdb_df.head()

,Unnamed: 0,title,original_title,popularity,vote_count,vote_average,poster_path
0,1,Manhattan Romance,Manhattan Romance,3.919,24,4.8,na
1,2,Stonehearst Asylum,Stonehearst Asylum,20.826,817,6.7,https://image.tmdb.org/t/p/w500/fZxGCCQ0NAtrae...
2,3,The Runner,The Runner,9.011,104,4.8,https://image.tmdb.org/t/p/w500/uIxFfrGlKEHJgk...
3,4,6 Years,6 Years,6.515,594,5.5,na
4,5,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,19.091,2689,6.4,https://image.tmdb.org/t/p/w500/vjnHRySiVM4YcB...


Transform Netflix and the Open Movie DB DataFrames

In [5]:
# Create a filtered on the Netflix dataframe from specific columns
netflix_columns = ["show_id", "title", "director", "cast", "release_year", "listed_in", "rating", "duration", "description"]
netflix_transformed= netflix_df[netflix_columns].copy()

# Rename the column headers
netflix_transformed = netflix_transformed.rename(columns={"show_id": "id",
                                                          "listed_in": "genre", 
                                                          "release_year": "year_released", 
                                                          "cast":"cast_members"})

# Clean the data by dropping duplicates and setting the index
netflix_transformed.drop_duplicates("id", inplace=True)
netflix_transformed.set_index("id", inplace=True)

netflix_transformed.head()

,title,director,cast_members,year_released,genre,rating,duration,description
id,,,,,,,,
80125979,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",2017,Comedies,TV-14,99 min,When nerdy high schooler Dani finally attracts...
80060297,Manhattan Romance,Tom O'Brien,"Tom O'Brien, Katherine Waterston, Caitlin Fitz...",2014,"Comedies, Independent Movies, Romantic Movies",TV-14,98 min,A filmmaker working on a documentary about lov...
70304988,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",2014,"Horror Movies, Thrillers",PG-13,113 min,"In 1899, a young doctor arrives at an asylum f..."
80057700,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",2015,"Dramas, Independent Movies",R,90 min,A New Orleans politician finds his idealistic ...
80045922,6 Years,Hannah Fidell,"Taissa Farmiga, Ben Rosenfield, Lindsay Burdge...",2015,"Dramas, Independent Movies, Romantic Movies",NR,80 min,As a volatile young couple who have been toget...


In [6]:
 # Create a filtered on the Open Move DB dataframe from specific columns
omdb_columns = ["title", "genre", "language", "awards", "production", "imdbid", "imdbrating"]
omdb_transformed= omdb_df[omdb_columns].copy()

# Clean the data by dropping duplicates and setting the index
omdb_transformed.drop_duplicates("imdbid", inplace=True)
omdb_transformed.set_index("imdbid", inplace=True)

omdb_transformed.head()

,title,genre,language,awards,production,imdbrating
imdbid,,,,,,
tt2608324,Manhattan Romance,"Comedy, Drama, Romance",English,3 wins.,Beacon Films Inc.,5.5
tt1772264,Stonehearst Asylum,"Drama, Horror, Thriller",English,1 win & 1 nomination.,Millenium Entertainment,6.8
tt3687398,The Runner,Drama,English,NaN,Alchemy,4.7
tt3799372,6 Years,"Drama, Romance",English,4 nominations.,NaN,5.6
tt3120280,Sierra Burgess Is a Loser,"Comedy, Drama, Romance",English,2 nominations.,Netflix,5.9


In [7]:
 # Create a filtered on the Open Move DB dataframe from specific columns
tmdb_columns = ["title", "original_title", "popularity", "vote_count", "vote_average", "poster_path"]
tmdb_transformed= tmdb_df[tmdb_columns].copy()

tmdb_transformed.head()

,title,original_title,popularity,vote_count,vote_average,poster_path
0,Manhattan Romance,Manhattan Romance,3.919,24,4.8,na
1,Stonehearst Asylum,Stonehearst Asylum,20.826,817,6.7,https://image.tmdb.org/t/p/w500/fZxGCCQ0NAtrae...
2,The Runner,The Runner,9.011,104,4.8,https://image.tmdb.org/t/p/w500/uIxFfrGlKEHJgk...
3,6 Years,6 Years,6.515,594,5.5,na
4,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,19.091,2689,6.4,https://image.tmdb.org/t/p/w500/vjnHRySiVM4YcB...


Create a DB connection

In [8]:
connection_string = "postgres:postgres@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Confirm tables were created
engine.table_names()

['netflix', 'tmdb', 'omdb']

Load Transformed DF data into the DB

In [12]:
netflix_transformed.to_sql(name='netflix', con=engine, if_exists='append', index=True)

In [13]:
omdb_transformed.to_sql(name='omdb', con=engine, if_exists='append', index=True)

In [16]:
tmdb_transformed.to_sql(name='tmdb', con=engine, if_exists='append', index=False)